In [7]:
import sys
print(sys.path)

from langchain.llms  import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import pandas as pd
import time

import os
import sys
import json
import openai

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

['/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/A-Github/lalossSQL/Entrega/OpenAI', '/Users/joaquinoldan/opt/anaconda3/envs/langchain/lib/python310.zip', '/Users/joaquinoldan/opt/anaconda3/envs/langchain/lib/python3.10', '/Users/joaquinoldan/opt/anaconda3/envs/langchain/lib/python3.10/lib-dynload', '', '/Users/joaquinoldan/opt/anaconda3/envs/langchain/lib/python3.10/site-packages']


In [8]:
OPENAI_API_KEY = 'ADD_YOUR_API_KEY_HERE'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

model_prices = [{'model':'gpt-4-1106-preview','input_price':10,'output_price':30},
                {'model':'gpt-3.5-turbo-0125','input_price':0.5,'output_price':1.5},
                {'model':'davinci-002','input_price':2,'output_price':2},
                {'model':'babbage-002','input_price':0.4,'output_price':0.4}]

client = openai.OpenAI()

#change model name to use different model
model_name = "gpt-3.5-turbo-0125"

#For davinci of babage use this generation function
def base_model_generation(prompt):
  response = openai.completions.create(
    model = model_name ,
    prompt=prompt,
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
      
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop = ["SQL"]
  ) 
  return response.choices[0].text, response.usage.prompt_tokens, response.usage.completion_tokens

#For GPT3.5 and GPT4 use this generation function
def GPT_generation(system_prompt, user_prompt):
  response = client.chat.completions.create(
    model= model_name ,
    messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": user_prompt}],
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
    
    frequency_penalty=0.0,
    presence_penalty=0.0,
  )
  return response.choices[0].message.content, response.usage.prompt_tokens, response.usage.completion_tokens



In [10]:
schema_linking_prompt = '''
Schema:
Table advisor, columns = [*,s_ID,i_ID]
Table classroom, columns = [*,building,room_number,capacity]
Table course, columns = [*,course_id,title,dept_name,credits]
Table department, columns = [*,dept_name,building,budget]
Table instructor, columns = [*,ID,name,dept_name,salary]
Table prereq, columns = [*,course_id,prereq_id]
Table section, columns = [*,course_id,sec_id,semester,year,building,room_number,time_slot_id]
Table student, columns = [*,ID,name,dept_name,tot_cred]
Table takes, columns = [*,ID,course_id,sec_id,semester,year,grade]
Table teaches, columns = [*,ID,course_id,sec_id,semester,year]
Table time_slot, columns = [*,time_slot_id,day,start_hr,start_min,end_hr,end_min]
Foreign_keys = [course.dept_name = department.dept_name,instructor.dept_name = department.dept_name,section.building = classroom.building,section.room_number = classroom.room_number,section.course_id = course.course_id,teaches.ID = instructor.ID,teaches.course_id = section.course_id,teaches.sec_id = section.sec_id,teaches.semester = section.semester,teaches.year = section.year,student.dept_name = department.dept_name,takes.ID = student.ID,takes.course_id = section.course_id,takes.sec_id = section.sec_id,takes.semester = section.semester,takes.year = section.year,advisor.s_ID = student.ID,advisor.i_ID = instructor.ID,prereq.prereq_id = course.course_id,prereq.course_id = course.course_id]
Q: "Find the buildings which have rooms with capacity more than 50."
A: Let’s think step by step. In the question "Find the buildings which have rooms with capacity more than 50.", we are asked:
"the buildings which have rooms" so we need table = [classroom] since we need the column = [classroom.capacity]
"rooms with capacity" so we need column = [classroom.building]
"rooms with capacity" so we need table = [classroom] since we need the column = [classroom.building]
Based on the tables, columns, and Foreign_keys. So the Tables_required are:
Tables_Required: ['classroom']

Schema:
Table department, columns = [*,Department_ID,Name,Creation,Ranking,Budget_in_Billions,Num_Employees]
Table head, columns = [*,head_ID,name,born_state,age]
Table management, columns = [*,department_ID,head_ID,temporary_acting]
Foreign_keys = [management.head_ID = head.head_ID,management.department_ID = department.Department_ID]
Q: "How many heads of the departments are older than 56 ?"
A: Let’s think step by step. In the question "How many heads of the departments are older than 56 ?", we are asked:
"How many heads of the departments" so we need table = [head] since we need the column = [head.*]
"older" than 56 so we need table = [head] since we need the column = [head.age]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['head']

Schema:
Table Country, columns = [*,id,name]
Table League, columns = [*,id,country_id,name]
Table Player, columns = [*,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight]
Table Player_Attributes, columns = [*,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes]
Table Team, columns = [*,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name]
Table Team_Attributes, columns = [*,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass]
Table sqlite_sequence, columns = [*,name,seq]
Foreign_keys = [Player_Attributes.player_api_id = Player.player_api_id,Player_Attributes.player_fifa_api_id = Player.player_fifa_api_id,League.country_id = Country.id,Team_Attributes.team_api_id = Team.team_api_id,Team_Attributes.team_fifa_api_id = Team.team_fifa_api_id]
Q: "List the names of all left-footed players who have overall rating between 85 and 90."
A: Let’s think step by step. In the question "List the names of all left-footed players who have overall rating between 85 and 90.", we are asked:
"names of all left-footed players" so we need table = [Player] since we need the column = [Player.player_name,Player_Attributes.preferred_foot]
"players who have overall rating" so we need table = [Player_Attributes] since we need column = [Player_Attributes.overall_rating]
Based on the tables, columns. So the Tables_required are:
Tables_Required: ['Player', 'Player_Attributes']
'''

In [13]:
DATASET_SCHEMA = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/spider/tables.json'
DATASET = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/spider/dev.json'
OUTPUT_FILE = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/A -Github/SQL_GPU/results'

def load_data(DATASET):
    return pd.read_json(DATASET)

import time

class TimeTracker:
    def __init__(self):
        # Initialize start_time and stop_time to None
        self.start_time = None
        self.stop_time = None

    def start(self):
        # Save the system time when the tracking starts
        self.start_time = time.time()

    def stop(self):
        # Save the system time when the tracking stops
        self.stop_time = time.time()

    def get_interval(self):
        # Calculate the time interval in seconds
        if self.start_time is not None and self.stop_time is not None:
            return self.stop_time - self.start_time
        else:
            return None

def schema_linking_prompt_maker(test_sample_text,database):
  instruction = "# Find the schema_links for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  fields = find_fields_MYSQL_like(database)
  foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  prompt = instruction + schema_linking_prompt + fields + 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""
  #prompt = instruction + schema_linking_prompt + fields +foreign_keys+ 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""  
  return prompt

def find_fields_MYSQL_like(db_name):
  df = spider_schema[spider_schema['Database name'] == db_name]
  df = df.groupby(' Table Name')
  output = ""
  for name, group in df:
    output += "Table " +name+ ', columns = ['
    for index, row in group.iterrows():
      output += row[" Field Name"]+','
    output = output[:-1]
    output += "]\n"
  return output

def find_foreign_keys_MYSQL_like(db_name):
  df = spider_foreign[spider_foreign['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ','
  output= output[:-1] + "]"
  return output

def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign


def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign

In [18]:
if __name__ == '__main__':
    spider_schema,spider_primary,spider_foreign = creatiing_schema(DATASET_SCHEMA)
    val_df = load_data(DATASET)
    print(f"Number of data samples {val_df.shape[0]}")
    CODEX = []
    count = 0
    for index, row in val_df.iterrows():
        print("")
        print(f"index is {index}")
        print(row['query'])
        print(row['question'])
        table_list = None
        error_encountered = False
        error=None
        while table_list is None:
            try:
                prompt = schema_linking_prompt_maker(row['question'], row['db_id'])
                #table_list = base_model_generation(prompt)

                table_list = GPT_generation("", prompt)
                print(table_list)
            except Exception as e:
                # Handling other exceptions
                print (e)
                error = e
                error_encountered = True
                pass
        if not error_encountered:
            try:
                table_list = table_list.split("Tables_Required: ")[1]
            except Exception as e:
                error = e
                error_encountered = True
                table_list = "[]"
            
        fields = find_fields_MYSQL_like(row['db_id'])
        foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(row['db_id']) + '\n'
        schema = fields + foreign_keys
        
        CODEX.append({'question':row['question'],'gold_query':row['query'], 'db_id': row['db_id'], 'schema':schema, 'table_list':table_list, 'error_encountered':error_encountered, 'error_type': type(error).__name__, 'error_message':str(error)})
        if index == 1: break
    
#    df = pd.DataFrame(CODEX, columns=['NLQ', 'PREDICTED SQL', 'GOLD SQL', 'DATABASE'])
#    results = df['PREDICTED SQL'].tolist()
#    with open(OUTPUT_FILE, 'w') as f:
#       for line in results:
#           f.write(f"{line}\n")

OUTPUT_FILE_DIFF = 'all-tables-GTP3.json'

OUTPUT_PATH = os.path.join(OUTPUT_FILE, OUTPUT_FILE_DIFF)

file_path = "output.json"

# Save the object to a JSON file


#with open(OUTPUT_PATH, 'w') as json_file:
#    json.dump(CODEX, json_file)

Number of data samples 1034

index is 0
SELECT count(*) FROM singer
How many singers do we have?
('In the question "How many singers do we have?", we are asked:\n"How many singers" so we need table = [singer] since we need the column = [singer.*]\nBased on the tables and columns provided, the Tables_required are:\nTables_Required: [\'singer\']', 1437, 60)

index is 1
SELECT count(*) FROM singer
What is the total number of singers?
('In the question "What is the total number of singers?", we are asked:\n"total number of singers" so we need table = [singer] since we need the column = [singer.*]\nBased on the tables and columns, the Table required is:\nTable_Required: [\'singer\']', 1438, 62)
